In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [13]:
%load_ext autoreload
%autoreload 2
from scripts.metrics.summary_evaluation import SummaryEvaluation
from scripts.models.rank_with_model.rank_with_model import RankModel
from scripts.models.baseline_rank.baseline_rank import BaselineRank
from scripts.models.ltr.train import LTRTrain
from scripts.extractive_summary.ltr.ltr_features_targets import LTRFeaturesTargets
from scripts.models.ltr_gbt.train import LTRGBTTrain
from scripts.models.ltr_svm_tf.train import LTRSVMTFTrain
from scripts import conf
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd

# Evaluación

Se usará el mismo desarrollo que se utilizó cuando se calcularon las métricas de los resúmenes generados por eventos clave y por grafos
conceptuales

# Baseline

## Carga de datos

In [5]:
key_events = ['goal', 'red_card', 'penalty']
lags = [1, 3, 5]
target_metric = 'cosine_tfidf'
drop_teams = True
lemma = True
metric_params = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [6]:
ltr = LTRFeaturesTargets(target_metric=target_metric, 
                        key_events=key_events,
                        lags=lags,
                        metric_params=metric_params,
                        count_vec_kwargs=count_vec_kwargs,
                        drop_teams=drop_teams,
                        lemma=lemma)

Setting target metric to cosine_tfidf


In [7]:
rank = BaselineRank(ltr=ltr, n=10)

Received LTR Features targets. Training and scoring are not necessary


In [23]:
rank.run()

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv


In [24]:
rank.ltr.target_metric

'cosine_tfidf'

## Evaluación

In [11]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [12]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


### Sin procesar

Los vocab size representan el número de frases que hay en la referencia. 

In [ ]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank,
                                                   preprocess_text=False)

### Con procesado

In [13]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/baseline_rank_cosine_tfidf_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/baseline_rank_cosine_tfidf_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/baseline_rank_cosine_tfidf_processed_avg.pickle


In [14]:
avg_scores_dict

8.631958094422498e-05

# Random forest

## Carga de datos

In [15]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'cosine_emb',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'embedding': 'distilroberta-base-paraphrase-v1',
                     'text_process': 'basic'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
num_features = ['tfidf_sum']
cat_features_dict = {'is_key_event': [0, 1],
                     'n_players_cat': ['no_player', 'one_player', 'more_than_one_player']}

model_params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [2, 5, 7],
    'bootstrap': [True],
    
}
cv = 5
opt_metric = 'neg_mean_squared_error'

In [20]:
ltr_train = LTRTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params)

Setting target metric to cosine_tfidf


In [16]:
#CV
ltr_train = LTRTrain(cat_features_dict=cat_features_dict,
                     num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to cosine_emb


In [17]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


## Evaluación

In [18]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [19]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


### Sin procesar

In [20]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                        preprocess_text=False)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/3829e2a65c/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b_avg.pickle


In [21]:
avg_scores_dict

0.00024863831147419623

### Con procesado

In [22]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/3829e2a65c/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b_processed_avg.pickle


In [23]:
avg_scores_dict

6.802229519430385e-05

# GBT

## Carga de datos

In [24]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'cosine_emb',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'embedding': 'distilroberta-base-paraphrase-v1',
                     'text_process': 'basic'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
cat_features_dict = {'is_key_event': [0, 1]}
num_features = ['tfidf_sum']
model_params = {
    'min_samples_leaf': [50],
    'subsample': [0.8],  
    'learning_rate': [0.1],
    'n_estimators': [80],
    'max_depth':[7],
    'min_samples_split': [600],
}
cv = 5
opt_metric = 'neg_mean_squared_error'

In [25]:
ltr_train = LTRGBTTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to cosine_emb


In [26]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [8]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/77752e3840'

## Evaluación

In [27]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [28]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


### Sin procesar

In [29]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/51f2ec7f23/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_avg.pickle


In [30]:
avg_scores_dict

0.0002504359209153971

### Con procesado

In [31]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/51f2ec7f23/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_processed_avg.pickle


In [32]:
avg_scores_dict

6.854264564757934e-05

# SVM

## Carga de datos

In [33]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'cosine_emb',
    'metric_params': {'embedding': 'distilroberta-base-paraphrase-v1',
                     'text_process': 'basic'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}

model_params = {}

In [34]:
ltr_train = LTRSVMTFTrain(model_params=model_params,
                          ltr_params=ltr_params,
                         dim_reduction_params=None)

Setting target metric to cosine_emb
Setting mode to tfidf


In [35]:
ltr_train = LTRSVMTFTrain(model_params=model_params, ltr_params=ltr_params,
                         dim_reduction_params=dim_reduction_params)

NameError: name 'dim_reduction_params' is not defined

In [ ]:
rank_model = RankModel(ltr_train, n=10)

In [8]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/c842b51b47'

## Evaluación

In [ ]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [ ]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

### Sin procesar

In [ ]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

In [ ]:
avg_scores_dict

### Con procesado

In [ ]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

In [ ]:
avg_scores_dict

# NN

## Carga de datos

In [ ]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'cosine_emb',
    'metric_params': {'embedding': 'distilroberta-base-paraphrase-v1',
                     'text_process': 'basic'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}
DENSE_LAYERS = [125, 60, 20]
N_DROPOUT = 3

model_params = {
    'dense_layers': DENSE_LAYERS,
    'dense_activation': ['relu'] * len(DENSE_LAYERS),
    'dropout': [0.3] * N_DROPOUT,
    'optimizer': 'adam',
}

dim_reduction_params = {
        'dim_reduction': 'truncated_svd',
        'dim_reduction_params': {
            'n_components': 250,
            'random_state': 10
    }
}

opt_metric = 'mean_squared_error'
epochs = 25
batch_size = 128
# Not in config
shuffle = True
max_queue_size = 50

In [ ]:
ltr_train = LTRNNTFTrain(ltr_params, epochs=epochs, batch_size=batch_size,
                         model_params=model_params,
                         opt_metric=opt_metric)

In [ ]:
ltr_train = LTRNNTFTrain(ltr_params, epochs=epochs, batch_size=batch_size,
                         model_params=model_params,
                         opt_metric=opt_metric, 
                        dim_reduction_params=dim_reduction_params)

In [ ]:
rank_model = RankModel(ltr_train, n=10, is_nn=True)

In [34]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/48e5a967da'

In [47]:
type(rank_model)

scripts.models.rank_with_model.rank_with_model.RankModel

## Evaluación

In [ ]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [ ]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

### Sin procesar

In [ ]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

In [ ]:
avg_scores_dict

### Con procesado

In [ ]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

In [ ]:
avg_scores_dict

# Eventos clave y grafos conceptuales

In [5]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [6]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


In [7]:
%%time
evaluation.evaluate_all_summaries()

Evaluating the following summaries: ['key_events_summaries_1.csv', 'key_events_summaries_3.csv', 'key_events_summaries_4.csv', 'key_events_summaries_graph_2.csv', 'key_events_summaries_graph_3.csv', 'key_events_summaries_2.csv', 'key_events_summaries_graph_5.csv', 'key_events_summaries_graph_1.csv', 'key_events_summaries_graph_4.csv']
Evaluating key_events_summaries_1.csv
Performing evaluation for 4302 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/key_events_summaries_1.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/key_events_summaries_1_avg.pickle
Evaluating key_events_summaries_3.csv
Performing evaluation for 4302 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/key_events_summaries_3.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/key_events_summaries_3_avg.pickle
Evaluating key_events_summaries_4.csv
Performing evaluation for 4300 articles
Writing to /home/carlos/MasterDS/tfm/

# Todos los resúmenes

In [14]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [15]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


In [16]:
pd_metrics = evaluation.output_avg_metrics()

In [17]:
pd_metrics

,experiment,score
0,key_events_summaries_1,0.000399
1,key_events_summaries_graph_1,0.000265
2,key_events_summaries_graph_2,0.000210
3,key_events_summaries_graph_3,0.000265
4,rank_with_model_ltr_random_forest_cosine_emb_1...,0.000068
5,key_events_summaries_4,0.000397
6,key_events_summaries_3,0.000399
7,key_events_summaries_graph_4,0.000210
8,rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_...,0.000069
9,rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80,0.000250
